## Add regions (continents)

Fill set information so we can start to do drilldown/drillup.

For this we add a members column that references other rows by primary key (code)

In [ ]:
df = pd.read_feather("intermediate/03-countries-with-entitiyids.feather")

In [ ]:
df.shape

❔ Do we have any rows that do not have a continent assigned in the continent column?

In [ ]:
df[df.continent.isnull()]

💡 It looks like Micronesia (region) is odd - do we have more entries like this?

In [ ]:
df[df.name.str.contains("icronesia")]

I would argue that "Rest of the World" and "Micronesia (region)" should be dropped. They have no other identifiers, no entitiy id and both are not clear enough that they could easily be used across table joins. If we get rid of those then the only entry left that will not have a continent assigned is "World" which sounds good

In [ ]:
df[df.continent.isnull() & df.code.isnull() & df.legacy_entity_id.isnull()]

In [ ]:
cleaned = df.drop(
    df.loc[
        df.continent.isnull() & df.code.isnull() & df.legacy_entity_id.isnull()
    ].index
)

In [ ]:
cleaned.shape

In [ ]:
cleaned[cleaned.code.isnull()]

⚡ Code will be our primary key and be mandatory for our table. Let's fill these in with some new, made up values.

In [ ]:
new_codes = {
    "Africa": "OWID_AFR",
    "Asia": "OWID_ASI",
    "Europe": "OWID_EUR",
    "Oceania": "OWID_OCE",
    "North America": "OWID_NAM",
    "Latin America": "OWID_LAM",
}

❔ Are any of these codes already used?

In [ ]:
cleaned[cleaned.code.isin(new_codes.values())]

In [ ]:
cleaned["code"] = cleaned.code.fillna(cleaned.name.map(new_codes))

In [ ]:
cleaned[cleaned.code.isnull()]

Make sure we don't have any duplicates in the code column

In [ ]:
cleaned.duplicated(subset=["code"]).any()

### Now group by continent, create and fill the members column

In [ ]:
continents_members = cleaned.groupby("continent")["code"].apply(list)

In [ ]:
continents_members

In [ ]:
continents_members.dtype

In [ ]:
# continents_members = continents_members.map(lambda m: pd.array(m, dtype="string"))

In [ ]:
continents_members[7.0]

In [ ]:
first_items = [l[0] for l in continents_members.values]

In [ ]:
cleaned[cleaned.code.isin(first_items)]

In [ ]:
continents_map = {
    1.0: "OWID_NAM",
    2.0: "OWID_ASI",
    3.0: "OWID_AFR",
    4.0: "OWID_EUR",
    5.0: "OWID_LAM",
    6.0: "OWID_OCE",
    7.0: "ATA",
}
reverse_continents_map = {key: value for (value, key) in continents_map.items()}

In [ ]:
cleaned["members"] = cleaned.code.map(
    lambda code: continents_members[reverse_continents_map[code]]
    if code in reverse_continents_map
    else []
)

In [ ]:
cleaned[cleaned.code == "ATA"]

In [ ]:
antarctica_index = cleaned[cleaned.code == "ATA"].index[0]

In [ ]:
antarctica_index

All looks good - let's just remove the self reference in Antarctica :)cleaned.loc[cleaned.code == "ATA", "members"].itemcleaned.at[cleaned.code == "ATA", "members"]

In [ ]:
without_ata = [
    m for m in cleaned.loc[cleaned.code == "ATA", "members"].item() if m != "ATA"
]

In [ ]:
without_ata

In [ ]:
cleaned.loc[cleaned.code == "ATA"]

In [ ]:
cleaned.at[antarctica_index, "members"] = without_ata

In [ ]:
cleaned.loc[cleaned.code == "ATA"]

Now set the world to have the continents as members

In [ ]:
cleaned.loc[cleaned.code == "OWID_WRL"]

In [ ]:
world_index = cleaned.loc[cleaned.code == "OWID_WRL"].index[0]

In [ ]:
continents_values = list(continents_map.values())

In [ ]:
continents_values

In [ ]:
cleaned.at[world_index, "members"] = continents_values

In [ ]:
cleaned.loc[cleaned.code == "OWID_WRL"]

## Save the file

In [ ]:
cleaned = cleaned.reset_index(drop=True)

In [ ]:
cleaned.dtypes

In [ ]:
cleaned_modern_dtypes = cleaned.convert_dtypes()

In [ ]:
cleaned_modern_dtypes.dtypes

In [ ]:
cleaned_modern_dtypes.head()

In [ ]:
cleaned_modern_dtypes.drop(["continent"], axis=1, inplace=True)

In [ ]:
cleaned_modern_dtypes.head()

In [ ]:
cleaned_modern_dtypes.loc[cleaned_modern_dtypes.code == "OWID_WRL"]

In [ ]:
import json

In [ ]:
cleaned_modern_dtypes.members = cleaned_modern_dtypes.members.map(
    lambda row: json.dumps(row) if row != [] else None
)

In [ ]:
cleaned_modern_dtypes.loc[cleaned_modern_dtypes.code == "OWID_WRL"]

In [ ]:
cleaned_modern_dtypes.head()

In [ ]:
cleaned_modern_dtypes.to_feather("intermediate/04-countries-with-continents.feather")